## Loading

In [34]:
from langchain_community.document_loaders import JSONLoader

In [35]:
# define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["source"] = record["metadata"]["source"]
    metadata["title"] = record["metadata"]["title"]
    return metadata

loader = JSONLoader(
    file_path="./news-sample1.json",
    jq_schema=".[]",
    content_key="page_content",
    metadata_func=metadata_func
)

In [36]:
data = loader.load()

## Splitting

In [37]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [38]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3500,
    chunk_overlap=0
)

In [39]:
docs = text_splitter.split_documents(data)

In [40]:
docs

[Document(metadata={'source': '핵심형', 'seq_num': 1, 'title': 'K'}, page_content='오랜시간, 주전, 필수적인, 핵심선수, 주축 선수, 에이스, 중심적인, 신뢰받는, 핵심 전력, 전술의 중심, 주전급, 책임감 있는, 핵심 선수, 불가결한, 리더형, 전략적 중심, 결정적인, 경기 지배력 있는, 압도적인, 팀의 중심, 경기 운영자, 영향력 있는, 반드시 필요한, 주요, 핵심키'),
 Document(metadata={'source': '조커형', 'seq_num': 2, 'title': 'J'}, page_content='교체선수, 적은시간, 조커, 슈퍼 서브, 임팩트 플레이어, 교체 카드, 후반전 강자, 분위기 반전, 한 방이 있는 선수, 기습적인, 결정적 순간의 카드, 경기 흐름 전환자, 단시간 해결사, 비장의 무기, 찬스를 만드는 선수, 후반 교체 요원, 짧고 강렬한, 유연한 전술 카드, 단기 해결사, 급격한, 영향력, 변칙적인, 정도환, 마무리, 피날래'),
 Document(metadata={'source': '소극형', 'seq_num': 3, 'title': 'I'}, page_content='소극적, 신중한, 조용한 플레이, 클린 플레이, 조심스러운, 소극적 수비, 자제력, 안전한 플레이, 침착한, 균형 잡힌 플레이, 규칙적인 플레이, 정교한 수비, 계산적인 움직임, 파울 회피, 안정적인 경기 운영, 예측 플레이, 영리한 수비, 인터셉트 중심, 과감함 부족, 점잖은 플레이, 수비 거리 유지, 중립적인 스타일, 패스 위주, 신사적인, 합리적인 경기 운영, 안정성 중시, 세련된 플레이, 파울 최소화, 이성적 판단, 규칙 준수, 감정 절제, 낮은 위험 감수, 무리하지 않는 플레이, 파울 유도, 위치 선정 중시, 피지컬 회피, 공을 지키는 플레이, 상대 자극 없음, 방어적인 경기 운영, 실리적인 스타일, 완급 조절, 필요 이상의 태클 회피, 패널티 박스 내 태클 자제, 두려움 없는 경기 부족, 침착

In [41]:
from langchain_openai import OpenAIEmbeddings

In [42]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

In [43]:
# Initialize and persist DB
import shutil
import os

DB_PATH = "./chroma_db"

# Ensure directory exists before removing
if os.path.exists(DB_PATH):
    shutil.rmtree(DB_PATH, ignore_errors=True)

os.makedirs(DB_PATH, exist_ok=True)


In [44]:
from langchain_chroma import Chroma

In [45]:
%%time

db = Chroma.from_documents(
    documents=docs,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)

CPU times: total: 328 ms
Wall time: 1.99 s


In [46]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# JSON 파일 로드
file_path = "./news-sample1.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# MBTI 비교할 쌍 정의 (A vs D, T vs P, E vs I, K vs J)
pairs = [("A", "D"), ("T", "P"), ("K", "J"), ("E", "I")]

# 쿼리 입력
query = "나는 전방에서 공격적인 포지션을 선호해 또한 여러 사람들과 주고받는 연계플레이를 좋아하고 팀에 헌신적인 스타일이야 비록 후반에 분위기 반전을 위해 교체선수로 많이 뛰긴 하지만 열정적인 스타일로 몸을 사리지 않는 저돌적이고 적극적인 스타일이야"

# 각 title에 해당하는 인덱스 매핑
title_to_index = {entry["metadata"]["title"]: idx for idx, entry in enumerate(data)}

# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
page_contents = [entry["page_content"] for entry in data] + [query]
tfidf_matrix = vectorizer.fit_transform(page_contents)

# 쿼리 벡터 가져오기
query_vector = tfidf_matrix[-1]  # 마지막 벡터가 query

# 비교 결과를 저장할 리스트
selected_titles = []

# 각 쌍을 비교하여 query와 더 유사한 항목 선택
for title1, title2 in pairs:
    if title1 not in title_to_index or title2 not in title_to_index:
        continue  # 해당 title이 데이터에 없으면 스킵

    idx1 = title_to_index[title1]
    idx2 = title_to_index[title2]

    similarity1 = cosine_similarity(query_vector, tfidf_matrix[idx1])[0][0]
    similarity2 = cosine_similarity(query_vector, tfidf_matrix[idx2])[0][0]

    # 쿼리와 더 유사도가 높은 문서 선택
    chosen_title = title1 if similarity1 > similarity2 else title2

    # 선택된 title을 리스트에 저장
    selected_titles.append(chosen_title)

# 결과 출력 (MBTI 순서대로 조합)
mbti_result = "".join(selected_titles)
print("MBTI 결과:", mbti_result)


MBTI 결과: APJI


In [47]:
import pandas as pd

# processed_player_data.csv 파일 불러오기
processed_file_path = './player.xlsx'
processed_data = pd.read_excel(processed_file_path)


# person_mbti와 'MBTI 코드'가 일치하는 선수들 필터링
filtered_data = processed_data[processed_data['MBTI 코드'] == mbti_result]

# 필터링된 데이터에서 '출전경기' 수가 가장 많은 3명 선수를 도출
top_3_players = filtered_data.nlargest(3, '경기')[['선수','팀','경기']]
top_3_players_str = top_3_players.to_string(index=False)

# 결과 출력
print(top_3_players)


           선수         팀  경기
5      크리스 우드  노팅엄 포레스트  31
179  킨 루이스-포터     브렌트퍼드  30
234    당고 와타라       본머스  30


In [48]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
# helper function for displaying markdown
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [49]:
model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0
)

In [50]:
top_3_players_str = top_3_players.to_string(index=False)

In [51]:
top_3_players_str

'      선수        팀  경기\n  크리스 우드 노팅엄 포레스트  31\n킨 루이스-포터    브렌트퍼드  30\n  당고 와타라      본머스  30'

In [53]:
chat_template1 = ChatPromptTemplate.from_messages(
                [
                    SystemMessage(
                        content=f"""당신은 축구 전문가이면서 심리행동전문가야.
                        사용자가 너에게 말하는 성향을 바탕으로 우리가 만든 축구 MBTI를 분석하고, 사용자에게 그 결과를 알려줘.
                        A(attack) 
                        D(defence) 
                        T(team)
                        P(personal)
                        K(key) 
                        J(joker)
                        E(extroversion)
                        I(Introverted)


                        이 순서대로 해당되는 영어에 따라 해당되는 4개를 설명해줘
                        
                        꼭 축구MBTI {mbti_result} 사용자에게 4가지 유형으로 제공해야 하고,  
                        정보가 부족해도 일단 사용자가 제공한 데이터로 알려줘!
                        결과에 대한 설명을 3줄 이상으로 길고 자세하게 풀어서 설명해줘.
                        마지막에는 사용자의 MBTI {mbti_result} 결과와 같은 축구 선수 3명을 추천해줘.

                        아래는 2023~2024 시즌 EPL리그에서 뛰었던 비슷한 유형의 선수 데이터야:
                        {top_3_players_str}
                        밑에 각 선수에 대한 소속팀, 포지션, 플레이스타일, 나이, 국적, 평가 등등 해당 선수에 대한 정보를 매우 구체적으로 알려줘
                        <예시>
                        ***2023~2024 시즌 EPL리그에서 뛰었던 비슷한 유형의 선수를 추천하겠습니다***
                        소속팀:
                        나이:
                        국적:
                        포지션:
                        평가:
                        """
                    ),
                    HumanMessage(content=query)
                ]
            )

In [54]:
messages1 = chat_template1.format_messages(text=query)

In [55]:
out1 = model.invoke(messages1)
printmd(out1.content)   

당신의 축구 MBTI는 APJI로 분석됩니다. 각 요소를 살펴보면 다음과 같습니다:

1. **A (Attack)**: 당신은 전방에서 공격적인 포지션을 선호한다고 하셨습니다. 이는 당신이 공격적인 플레이를 통해 팀의 득점을 이끌어내고 싶어하는 성향을 나타냅니다. 공격적인 플레이는 상대 수비를 압박하고, 기회를 창출하는 데 중요한 역할을 합니다.

2. **P (Personal)**: 여러 사람들과의 연계 플레이를 좋아하고 팀에 헌신적인 스타일이라고 하셨습니다. 이는 개인적인 성향이 강하면서도 팀워크를 중시하는 것을 의미합니다. 당신은 팀의 성공을 위해 개인의 기량을 발휘하는 것을 중요하게 생각하며, 동료들과의 협력을 통해 더 나은 결과를 만들어내고자 합니다.

3. **J (Joker)**: 후반에 분위기 반전을 위해 교체 선수로 많이 뛰는 스타일은 당신이 상황에 따라 유연하게 대처할 수 있는 능력을 가지고 있음을 나타냅니다. 이는 팀의 전술에 맞춰 적절한 타이밍에 투입되어 경기를 변화시키는 역할을 잘 수행할 수 있다는 것을 의미합니다.

4. **I (Introverted)**: 저돌적이고 적극적인 스타일이지만, 팀 내에서의 역할을 중시하는 점에서 내향적인 성향이 엿보입니다. 이는 당신이 팀의 일원으로서 조화를 이루는 것을 중요하게 생각하며, 개인적인 성향이 팀의 목표와 잘 맞아떨어진다는 것을 보여줍니다.

이러한 성향을 바탕으로, 당신은 팀의 공격을 이끌며, 동료들과의 협력을 통해 경기를 주도하는 역할을 잘 수행할 수 있는 선수입니다. 당신의 MBTI APJI 결과와 비슷한 유형의 선수로는 다음과 같은 선수들을 추천합니다:

***2023~2024 시즌 EPL리그에서 뛰었던 비슷한 유형의 선수를 추천하겠습니다***

1. **크리스 우드**
   - 소속팀: 노팅엄 포레스트
   - 나이: 31세
   - 국적: 뉴질랜드
   - 포지션: 공격수
   - 평가: 강력한 피지컬과 공중볼 장악 능력을 바탕으로 팀의 공격을 이끌며, 연계 플레이에서도 중요한 역할을 수행합니다.

2. **킨 루이스-포터**
   - 소속팀: 브렌트퍼드
   - 나이: 30세
   - 국적: 잉글랜드
   - 포지션: 공격수
   - 평가: 빠른 스피드와 기술적인 드리블로 상대 수비를 흔들며, 팀의 공격 전개에 기여하는 스타일입니다.

3. **당고 와타라**
   - 소속팀: 본머스
   - 나이: 22세
   - 국적: 부르키나파소
   - 포지션: 공격수
   - 평가: 젊은 나이에 비해 뛰어난 기량을 보유하고 있으며, 팀의 공격을 이끌며 다양한 포지션에서 활약할 수 있는 유연성을 가지고 있습니다.

이 선수들은 당신의 성향과 비슷한 스타일로, 팀의 공격을 이끌며 연계 플레이에 능한 선수들입니다.